In [1]:
import calliope
import pandas as pd
from building import Building
import geopandas as gpd
import gc
import os
import matplotlib.pyplot as plt
# import glob
import warnings
import numpy as np
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
from matplotlib.colorbar import ColorbarBase

c:\Users\wangy\Documents\GitHub\calliope_optimization\building.py:214: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

c:\Users\wangy\Documents\GitHub\calliope_optimization\building.py:293: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



In [2]:
calliope.set_log_verbosity(verbosity='error', include_solver_output=False, capture_warnings=False)
# disable the warnings
warnings.filterwarnings('ignore')

In [3]:
def get_df_from_dbf(dbf_path: str) -> pd.DataFrame:
    """
    Description: 
    The .dbf file is fully translated into this dataframe. 
    All datatypes are indicated automatically by pandas, thus needs manual modification later if needed.
    Since the dbf contains all building's parameter generated by CEA, 'Name' column is set as index.
    
    Inputs:
    dbf_path:   str, path of .dbf file

    Return:
    df:         pd.DataFrame, a dataframe indexed by building names. 
    """

    df: pd.DataFrame = gpd.read_file(dbf_path, ignore_geometry=True) # if ignore_geometry == True, then return a pd.DataFrame
    df.set_index("Name", inplace=True)
    return df

In [4]:
scenario_folder = r'C:\Users\wangy\OneDrive\ETHY2FW\IDP_Personal\CEA\2050 w3'
zone_gdf = gpd.read_file(scenario_folder+r'\inputs\building-geometry\zone.shp')
zone_gdf.index = zone_gdf['Name']
# create a new df using only the columns we need
zone_df = pd.DataFrame()
zone_df['Name'] = zone_gdf['Name']
zone_df.set_index('Name', inplace=True)
# # delete the Name column in zone_df
# # typology_df = get_df_from_dbf(scenario_folder+r'\inputs\building-properties\typology.dbf')
# # zone_df = pd.concat([zone_df, typology_df['YEAR'], ], axis=1)
# # zone_gdf['area'] = zone_gdf['geometry'].area
# # zone_df = pd.concat([zone_df, zone_gdf['area'], zone_gdf['floors_ag']], axis=1)
# # read typology file 
# # read the csv file containing if the building is inside district area
# district_df = pd.read_csv(scenario_folder+r'\inputs\is_disheat.csv', index_col=0)
# # merge the two df
# zone_df = zone_df.merge(district_df, left_index=True, right_index=True)
# # read the csv file containing if building is rebuilt
# rebuild_df = pd.read_csv(scenario_folder+r'\inputs\Rebuild.csv', index_col=0)
# # merge the two df, but since the zone_df has more columns, if rebuild_df doesn't contain some building, set the value to 0
# zone_df = zone_df.merge(rebuild_df, left_index=True, right_index=True, how='left').fillna(0)
# # read the csv file containing if the building is renovated
# renovate_df = pd.read_csv(scenario_folder+r'\inputs\Renovation.csv', index_col=0)
# # merge the two df, but since the zone_df has more columns, if renovate_df doesn't contain some building, set the value to 0
# zone_df = zone_df.merge(renovate_df, left_index=True, right_index=True, how='left').fillna(0)
# # read dbf file containing current building systems
# current_heating_system_df = get_df_from_dbf(r"C:\Users\wangy\OneDrive\ETHY2FW\IDP_Personal\CEA\2023 Baseline\inputs\building-properties\supply_systems.dbf")[['type_hs']]
# # leave only the type_hs column
# # current_system_df = current_system_df[['type_hs']]
# # merge the two df, but since the zone_df has more columns
# zone_df = zone_df.merge(current_heating_system_df, left_index=True, right_index=True, how='left')

In [5]:
# # add indicating columns to guide adjusting AttrDict based on building's status
# zone_df['is_disheat'] = zone_df['DisHeat'].astype(bool)
# zone_df['is_rebuilt'] = zone_df['Rebuild'].astype(bool)
# zone_df['is_renovated'] = zone_df['Renovation'].astype(bool)
# zone_df['is_new'] = ~zone_df['is_rebuilt'] & ~zone_df['is_renovated']
# zone_df['already_GSHP'] = zone_df['type_hs'] == 'SUPPLY_HEATING_AS6'
# zone_df['already_ASHP'] = zone_df['type_hs'] == 'SUPPLY_HEATING_AS7'
# zone_df['no_heat'] = zone_df['type_hs'] == 'SUPPLY_HEATING_AS0'

In [6]:
# # get all dbfs from the following subfolder
# dbf_folder = scenario_folder + r"\inputs\building-properties"
# dbf_files = os.listdir(dbf_folder)
# for file in dbf_files:
#     if file.endswith(".dbf"):
#         dbf_path = dbf_folder + "\\" + file
#         df = get_df_from_dbf(dbf_path)
#         # set the prefix of the column name to the name of the dbf file
#         df.columns = [file[:-4] + "_" + col for col in df.columns]
#         zone_df = pd.concat([zone_df, df], axis=1)

In [7]:
# iterate through the zone_df to create the building objects and store them in a list
store_folder = './pareto_no_wood'
# check if building's name is already stored in subfolder result_per_building; if not, add it to the list
for index, row in zone_df.iterrows():
    building_name = str(index)
    if building_name+'_pareto.csv' in os.listdir(store_folder):
        print(building_name+' is already done')
        continue
    elif row["no_heat"] == 1:
        print(building_name+' doesn\'t need heating')
        continue
    else: # building is not yet optimized, continue
        pass

    building = Building(name=building_name, scenario_path=scenario_folder, calliope_yaml_path='./data/technology/techs_plot8_24h.yml')
    building.get_pareto_front(epsilon=3, 
                              store_folder=store_folder,
                              building_status=row, flatten_spikes=True, flatten_percentile=0.98, 
                              to_lp=False, to_yaml=False)
    building.df_pareto.to_csv(store_folder+'/'+building_name+'_pareto.csv')
    
    

B162953 is already done
B162948 is already done
B162947 is already done
B302011886 doesn't need heating
B162917 is already done
B2365661 is already done
B162952 is already done
B302012803 doesn't need heating
B162935 is already done
B162946 is already done
B302024523 is already done
B302024094 doesn't need heating
B302024524 is already done
B302013071 doesn't need heating
B302024327 doesn't need heating
B162920 is already done
B302011676 doesn't need heating
B302060757 doesn't need heating
B162916 is already done
B302022562 is already done
B162581 is already done
B302022561 is already done
B162585 is already done
B162579 is already done
B162578 is already done
B302024344 doesn't need heating
B302011740 doesn't need heating
B162396 is already done
B162831 is already done
B162482 is already done
B162404 is already done
B162446 is already done
B302024101 doesn't need heating
B162493 is already done
B162930 is already done
B162327 is already done
B162510 is already done
B2365747 is already

ApplicationError: No executable found for solver 'cplex'

In [ ]:
# iterate through the zone_df get pareto front for each building and store output in a dictionary
for building in building_list:
    # if building doesn't need heating, skip it
    if zone_df[building.name, "no_heat"] == 1:
        print(building.name+' doesn\'t need heating')
        continue
    
    # if building is not in district heating area, skip it
    building.get_pareto_front()
    # save the df_pareto as csv
    building.df_pareto.to_csv(store_folder+'/'+idx+'_pareto.csv')
    del df_pareto
    print(idx+' is done')
    # clean up the rest of the memory
    gc.collect()

## Plotting

In [ ]:
# plot setting
# set style of plots
plt.rcParams['font.family'] = 'Roboto'
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['legend.loc'] = 'lower center'
plt.rcParams['legend.frameon'] = False
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.titley'] = 1.03
plt.rcParams['figure.subplot.left'] = 0.08
plt.rcParams['figure.subplot.right'] = 0.95
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['xtick.major.width'] = 2  # setting the x-axis tick width globally
plt.rcParams['ytick.major.width'] = 2 # setting the y-axis tick width globally
# set fig size
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
# calculate the coefficient of variation of each technology choice for each building and store them in a dataframe; in the end plot a bar chart with error bars for each technology in this dataframe.
# the dataframe should have columns as technology names, and rows as building names
# first define the dataframe column from one of the files in the folder;
# then iterate through the rest of the files and add the data to the dataframe
ls_csv = []
for csv in os.listdir(store_folder):
    if not csv.endswith('.csv'):
        continue
    ls_csv.append(csv)

for idx, csv in enumerate(ls_csv):
    # if the dataframe doesn't exist before, then define its columns with the first file
    if idx == 0:
        df_example = pd.read_csv('./pareto_no_wood/'+csv, index_col=0, nrows=1)
        # add the first level index to be building name
        df_example.index = pd.MultiIndex.from_tuples([(csv, i) for i in df_example.index], names=['building', 'epsilon'])
        df_tech_cap = pd.DataFrame(columns=df_example.columns, index=df_example.index)
        df_tech_cap = df_tech_cap.dropna(how='all')

    building_sizing = pd.read_csv(store_folder+'/'+csv, index_col=0)
    # # calculate the coefficient of variation for each technology
    # df_tech_cap.loc[csv[:-10]] = building_sizing.std() / building_sizing.mean()
    building_name = csv[:-11]
    # if wood_boiler_DHW is 2000 at emission optimal (the first row), then it's oversized, change it to the DHW demand
    if building_sizing.loc[0, 'wood_boiler_DHW'] == 2000:
        building_sizing.loc[0, 'wood_boiler_DHW'] = building_sizing.loc[4, 'demand_hot_water']
        building_sizing.loc[0, 'DHDC_small_heat'] = 0
        building_sizing.loc[0, 'DHDC_medium_heat'] = 0
        building_sizing.loc[0, 'DHDC_large_heat'] = 0
    # if wood_boiler_heat is 2000 at emission optimal (the first row), then it's oversized, change it to the SH demand
    if building_sizing.loc[0, 'wood_boiler_heat'] == 2000:
        building_sizing.loc[0, 'wood_boiler_heat'] = building_sizing.loc[4, 'demand_space_heating']
        
    building_area = zone_df.loc[building_name, 'area'] * zone_df.loc[building_name, 'floors_ag']
    # calculate the sizing per m2
    building_sizing_per_m2 = building_sizing.fillna(0) / building_area
    # add the first level index to be building name
    building_sizing_per_m2.reset_index(drop=True, inplace=True)
    building_sizing_per_m2.index = pd.MultiIndex.from_product([[building_name], building_sizing_per_m2.index], names=['building', 'epsilon'])
    
    # add the building_sizing_per_m2 to df_tech_cap to the bottom, but keep the multiindex
    df_tech_cap = pd.concat([df_tech_cap, building_sizing_per_m2], axis=0)


In [ ]:
# check if df_tech_cap has two levels of index
df_tech_cap.index.nlevels
# filter buildings that have GSHP. Refer to zone_df['already_GSHP'] for the boolean value, and refresh the df_tech_cap
df_tech_cap = df_tech_cap[df_tech_cap.index.get_level_values('building').isin(zone_df[zone_df['is_disheat']].index)]

In [ ]:
year_min = zone_df['YEAR'].min()
year_max = zone_df['YEAR'].max()
year_norm = mcolors.Normalize(vmin=year_min, vmax=year_max)
colormap = plt.cm.get_cmap('viridis')
for building_name in df_tech_cap.index.get_level_values('building').unique():
    # get the df_pareto for this building
    df_pareto = df_tech_cap.loc[building_name]
    # find the building area
    building_area = zone_df.loc[building_name, 'area'] * zone_df.loc[building_name, 'floors_ag']
    # plot the pareto front, and matching the curve's color in colormap by building's year
    building_year = zone_df.loc[building_name, 'YEAR']
    building_color = colormap(year_norm(building_year))
    # ignore the mission optimal point which is the first point
    plt.plot(df_pareto['emission'].iloc[1:], 
             df_pareto['cost'].iloc[1:], 
             label=building_name, marker='.', color=building_color, linewidth=0.75)
    
plt.ylabel('cost [$CHF/m^2$]')
plt.xlabel('emission [$kg/m^2$]')
plt.title('Pareto Front, normalized by GFA', fontsize=24)
plt.ylim(7, 65)
plt.xlim(3, 26)
# plt.xlim(1.5, 13)
# plt.ylim(0, 50)
# create a colorbar next to the plot, with the same height as the plot box
cax = plt.axes([0.96, 0.11, 0.02, 0.77])
cbar = ColorbarBase(cax, cmap=colormap, norm=year_norm)
# label the building year on the left of the colorbar
cbar.ax.set_ylabel('year of built')
# plt.savefig(r'C:\Users\wangy\polybox - Yiqiao Wang (yiqwang@student.ethz.ch)@polybox.ethz.ch\Final Report\Graphics\ParetoFront_{}.pdf'.format(store_folder[9:]), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(5,8))
for building_name in df_tech_cap.index.get_level_values('building').unique():
    # get the df_pareto for this building
    df_pareto = df_tech_cap.loc[building_name]
    # the first point is emission minimal, thus cost maximal; 
    # last one is cost minimal, thus emission maximal
    # the slope defines how hard it is to reduce emission when sliding from cost minimal to emission minimal
    y1 = df_pareto['cost'].iloc[0]
    y2 = df_pareto['cost'].iloc[3]
    x1 = df_pareto['emission'].iloc[0]
    x2 = df_pareto['emission'].iloc[3]
    emission_reduction_relative = (x2-x1)/x1*100 # percentage of emission reduction
    cost_increase_absolute = y1-y2 # absolute cost increase
    # label the points with building name next to the points
    building_year = zone_df.loc[building_name, 'YEAR']
    # building_area = zone_df.loc[building_name, 'area'] * zone_df.loc[building_name, 'floors_ag']
    building_color = colormap(year_norm(building_year))
    # find building typology 
    plt.scatter(cost_increase_absolute, emission_reduction_relative, label=building_name, marker='o', color=building_color)
    # annotate the building name next to the point
    if zone_df.loc[building_name, 'typology_1ST_USE'] not in ['SINGLE_RES', 'MULTI_RES']:
        plt.annotate(zone_df.loc[building_name, 'typology_1ST_USE'], (cost_increase_absolute, emission_reduction_relative), 
                     fontsize=8, xytext=(0, 5), textcoords='offset points', ha='left')
    
plt.ylabel('Percentage of emission reduction [%]')
plt.xlabel('Additional cost to reduce emission [$CHF/m^2$]')
plt.xlim(0, 150)
plt.ylim(0, 40)
# plot a line from (0,0) to (100, 1) and extend to the right
plt.plot([0, 75], [0, 40], color='k', linestyle='--', linewidth=0.5)
plt.plot([0, 150], [0, 40], color='k', linestyle='--', linewidth=0.5)
plt.plot([0, 250], [0, 40], color='k', linestyle='--', linewidth=0.5)
plt.plot([0, 400], [0, 40], color='k', linestyle='--', linewidth=0.5)

# plot a horizontal line at 800
# plt.axhline(y=900, color='k', linestyle='--', linewidth=0.5)
# plt.axhline(y=530, color='k', linestyle='--', linewidth=0.5)
plt.title('Fraction of total emission reduction', fontsize=24)

# again, plot a colorbar next to the plot, with the same height as the plot box
cax = plt.axes([0.96, 0.11, 0.02, 0.77])
cbar = ColorbarBase(cax, cmap=colormap, norm=year_norm)
# label the building year on the left of the colorbar
cbar.ax.set_ylabel('year of built')

plt.savefig(r'C:\Users\wangy\polybox - Yiqiao Wang (yiqwang@student.ethz.ch)@polybox.ethz.ch\Final Report\Graphics\relative_{}_thinner.pdf'.format(store_folder[9:]), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(5,8))
for building_name in df_tech_cap.index.get_level_values('building').unique():
    # get the df_pareto for this building
    df_pareto = df_tech_cap.loc[building_name]
    # the first point is emission minimal, thus cost maximal; 
    # last one is cost minimal, thus emission maximal
    # the slope defines how hard it is to reduce emission when sliding from cost minimal to emission minimal
    y1 = df_pareto['cost'].iloc[0]
    y2 = df_pareto['cost'].iloc[3]
    x1 = df_pareto['emission'].iloc[0]
    x2 = df_pareto['emission'].iloc[3]
    emission_reduction_absolute = x2-x1 # percentage of emission reduction
    cost_increase_absolute = y1-y2 # absolute cost increase
    # label the points with building name next to the points
    building_year = zone_df.loc[building_name, 'YEAR']
    # building_area = zone_df.loc[building_name, 'area'] * zone_df.loc[building_name, 'floors_ag']
    building_color = colormap(year_norm(building_year))
    # find building typology 
    plt.scatter(cost_increase_absolute, emission_reduction_absolute, label=building_name, marker='o', color=building_color)
    # annotate the building name next to the point
    if zone_df.loc[building_name, 'typology_1ST_USE'] not in ['SINGLE_RES', 'MULTI_RES']:
        plt.annotate(zone_df.loc[building_name, 'typology_1ST_USE'], (cost_increase_absolute, emission_reduction_absolute), 
                     fontsize=8, xytext=(0, 5), textcoords='offset points', ha='left')
    
plt.ylabel('Absolute emission reduction [$kg/m^2$]')
plt.xlabel('Additional cost to reduce emission [$CHF/m^2$]')
plt.xlim(0, 150)
plt.ylim(0, 2.25)
# set y ticks to tick only every 0.5
plt.yticks(np.arange(0, 2.25, 0.5))
# plot a line from (0,0) to (100, 1) and extend to the right
plt.plot([0, 75], [0, 2.25], color='k', linestyle='--', linewidth=0.5)
plt.plot([0, 150], [0, 2.25], color='k', linestyle='--', linewidth=0.5)
plt.plot([0, 250], [0, 2.25], color='k', linestyle='--', linewidth=0.5)
plt.plot([0, 400], [0, 2.25], color='k', linestyle='--', linewidth=0.5)

# plot a horizontal line at 800
# plt.axhline(y=900, color='k', linestyle='--', linewidth=0.5)
# plt.axhline(y=530, color='k', linestyle='--', linewidth=0.5)
plt.title('Total emission reduction', fontsize=24)

# again, plot a colorbar next to the plot, with the same height as the plot box
cax = plt.axes([0.96, 0.11, 0.02, 0.77])
cbar = ColorbarBase(cax, cmap=colormap, norm=year_norm)
# label the building year on the left of the colorbar
cbar.ax.set_ylabel('year of built')

plt.savefig(r'C:\Users\wangy\polybox - Yiqiao Wang (yiqwang@student.ethz.ch)@polybox.ethz.ch\Final Report\Graphics\absolute_{}_thinner.pdf'.format(store_folder[9:]), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
for building_name in df_tech_cap.index.get_level_values('building').unique():
    # get the df_pareto for this building
    df_pareto = df_tech_cap.loc[building_name]
    # the first point is emission minimal, thus cost maximal; 
    # last one is cost minimal, thus emission maximal
    # the slope defines how hard it is to reduce emission when sliding from cost minimal to emission minimal
    y1 = df_pareto['cost'].iloc[1]
    y2 = df_pareto['cost'].iloc[4]
    x1 = df_pareto['emission'].iloc[1]
    x2 = df_pareto['emission'].iloc[4]
    slope = -(y2-y1)/(x2-x1)
    average_emission = (df_pareto['emission'].iloc[-3] + df_pareto['emission'].iloc[1]) / 2
    # label the points with building name next to the points
    building_year = zone_df.loc[building_name, 'YEAR']
    # building_area = zone_df.loc[building_name, 'area'] * zone_df.loc[building_name, 'floors_ag']
    building_color = colormap(year_norm(building_year))
    # find building typology 
    plt.scatter(average_emission, slope, label=building_name, marker='o', color=building_color)
    # annotate the building name next to the point
    if zone_df.loc[building_name, 'typology_1ST_USE'] not in ['SINGLE_RES', 'MULTI_RES']:
        plt.annotate(zone_df.loc[building_name, 'typology_1ST_USE'], (average_emission, slope), 
                     fontsize=8, xytext=(0, 5), textcoords='offset points', ha='left')
    # add name of the building next to the point
    # plt.annotate(building_name, (average_emission, slope), fontsize=8, xytext=(0, 5), textcoords='offset points', ha='left')
    
plt.ylabel('Elasticity of cost to emission [-]')
plt.xlabel('Average emission [$kg/m^2$]')
# plt.xlim(3, 26)
# plt.ylim(1.8, 4.9)
# plot a horizontal line at 800
# plt.axhline(y=900, color='k', linestyle='--', linewidth=0.5)
# plt.axhline(y=530, color='k', linestyle='--', linewidth=0.5)
plt.title('Difficulties for buildings to reduce emission', fontsize=24)

# again, plot a colorbar next to the plot, with the same height as the plot box
cax = plt.axes([0.96, 0.11, 0.02, 0.77])
cbar = ColorbarBase(cax, cmap=colormap, norm=year_norm)
# label the building year on the left of the colorbar
cbar.ax.set_ylabel('year of built')

# plt.savefig(r'C:\Users\wangy\polybox - Yiqiao Wang (yiqwang@student.ethz.ch)@polybox.ethz.ch\Final Report\Graphics\ParetoFront_elasticity_{}.pdf'.format(store_folder[9:]), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Separate the demand technologies
df_demand_tech = df_tech_cap[df_tech_cap.columns[-4:]]
df_supply_tech = df_tech_cap[df_tech_cap.columns[:-4]]

# Create a figure with two subplots side by side, with a shared y-axis and width ratio of 3:1
fig, axs = plt.subplots(1, 2, sharey=True,figsize=(20, 6), gridspec_kw={'width_ratios': [4, 1]})

# Boxplot without outliers and with a custom hatch pattern
boxprops = dict(linestyle='-', linewidth=1, color='k')  # Custom box properties
medianprops = dict(linestyle='-', linewidth=0.5, color='k')  # Custom median properties

# Create a color map for the five epsilon cuts
cmap = plt.cm.get_cmap('RdYlGn', 5)# 5 distinct colors for 5 epsilon cuts
# reverse the color map so that the emission optimal is green
cmap = cmap.reversed()

# Create a boxplot for each technology
for i, tech in enumerate(df_supply_tech.columns):
    # Get the data for the current technology
    tech_data = df_supply_tech[tech].values.reshape(-1, 5)  # Reshape the data to have 5 rows for each technology
    bp = axs[0].boxplot(tech_data, positions=np.arange(i*5, i*5+5), patch_artist=True, showfliers=False, boxprops=boxprops, medianprops=medianprops)

    # Fill each box with a color according to its epsilon cut
    for j, box in enumerate(bp['boxes']):
        box.set(facecolor=cmap(j))

# Define your colors for the demand technologies
demand_colors = ['tab:green', 'tab:orange', 'tab:blue', 'tab:red']

# Create a boxplot for each demand technology, but only one box per technology
for i, tech in enumerate(df_demand_tech.columns):
    # calculate the mean of each technology in five epsilon cuts
    tech_data = df_demand_tech[tech].values.reshape(-1, 5).mean(axis=1)
    bp = axs[1].boxplot(tech_data, positions=[i], patch_artist=True, showfliers=False, boxprops=boxprops, medianprops=medianprops, widths=0.5)  # Set position to a single value and adjust the width
    # Fill each box with a color according to its epsilon cut
    for j, box in enumerate(bp['boxes']):
        box.set(facecolor=demand_colors[i])

# Set the x-axis labels to the technology names
axs[0].set_xticks(np.arange(2.5, len(df_supply_tech.columns)*5, 5))  # Set x-ticks to the middle box of each technology group
axs[0].set_xticklabels(df_supply_tech.columns, rotation=45)

# Align the x-axis labels to the right
for label in axs[0].get_xticklabels():
    label.set_horizontalalignment('right')

# Draw vertical lines to separate the technologies inbetween the boundary boxes, not on the box
for i in range(5, len(df_supply_tech.columns)*5, 5):
    axs[0].axvline(x=i-0.5, color='k', linestyle='--', linewidth=0.5)  # Subtract 0.5 from the x position

# set the second subplot's x axis to be the same as the first one, but only one box per technology
axs[1].set_xticks(range(len(df_demand_tech.columns)))  # Set x-ticks to the middle box of each technology group
axs[1].set_xticklabels(df_demand_tech.columns, rotation=45)

for label in axs[1].get_xticklabels():
    label.set_horizontalalignment('right')
# Set the title and axis labels
axs[0].set_title('Variation of Building Technologies', fontsize=24)
axs[1].set_title('Variation of Demand', fontsize=24)
axs[0].set_ylabel('Specific Sizing of Technologies [$kW/m^2$]', fontsize=14)

# Create a custom legend for the epsilon cuts
ls_epsilon_cut = ['Emission Optimal', 'Epsilon 1', 'Epsilon 2', 'Epsilon 3', 'Cost Optimal']
legend_patches = [mpatches.Patch(color=cmap(i), label=ls_epsilon_cut[i]) for i in range(5)]
axs[0].legend(handles=legend_patches, loc='upper left')

# print a horizontal line at 0 for both subplots
axs[0].axhline(y=0, color='k', linestyle='--', linewidth=0.5)
axs[1].axhline(y=0, color='k', linestyle='--', linewidth=0.5)

# Set the y-axis limits
axs[0].set_ylim(-0.005, 0.09)
# make the two plots a bit closer
plt.subplots_adjust(wspace=0.05)
# Tight layout to adjust padding
# plt.tight_layout()

plt.savefig(r'C:\Users\wangy\polybox - Yiqiao Wang (yiqwang@student.ethz.ch)@polybox.ethz.ch\Final Report\Graphics\Variation_{}_shorter.pdf'.format(store_folder[9:]), dpi=300, bbox_inches='tight')
# Show the plot
plt.show()

## explore different clustering of building against pareto front
1. The goal is to compare buildings with different DH capex. If five buildings are using the same DH outlet, then every building should only share 1/5 of the DH capex. 
2. There are some buildings that belongs to the same developer. These buildings can be clustered and share the DH capex. Below I will list the buildings that belongs to the same developer.

## test for one building

In [ ]:
building = Building('B302030807', scenario_folder)
building.flatten_spikes_demand(percentile=0.98) # flatten the demand spikes
dict_timeseries_df = {'demand_el': building.app,
                      'demand_sh': building.sh,
                      'demand_dhw': building.dhw,
                      'demand_sc': building.sc,
                      'supply_PV': building.pv_intensity,
                      'supply_PVT_e': building.pvt_e_intensity,
                      'supply_PVT_h': building.pvt_h_relative_intensity,
                      'supply_SCFP': building.scfp_intensity
                      }
row = zone_df.loc[building.name]
# modify the building_specific_config to match the building's status
building_specific_config: calliope.AttrDict = calliope.AttrDict.from_yaml('./data/technology/techs_plot8.yml')
building_specific_config.set_key(key='locations.Building.available_area', value=building.area)
print('the area of building '+building.name+' is '+str(building.area)+' m2')
building_sub_dict = building_specific_config['locations'].pop('Building')
building_specific_config['locations'][building.name] = building_sub_dict

# update geothermal storage max capacity
building_specific_config.set_key(key=f'locations.{building.name}.techs.geothermal_boreholes.constraints.energy_cap_max',
                                 value=(building.area+400)*0.1) # assume 100W/m2 max yield
# if building is not in district heating area, delete the district heating technologies keys
if not row['is_disheat']:
    building_specific_config.del_key(f'locations.{building.name}.techs.DHDC_small_heat')
    building_specific_config.del_key(f'locations.{building.name}.techs.DHDC_medium_heat')
    building_specific_config.del_key(f'locations.{building.name}.techs.DHDC_large_heat')

# if building is already equipped with GSHP, set the GSHP purchase cost to 0
if row['already_GSHP']:
    building_specific_config.set_key(f'locations.{building.name}.techs.GSHP_heat.costs.monetary.purchase', 0)
    building_specific_config.set_key(f'locations.{building.name}.techs.GSHP_heat.costs.monetary.energy_cap', 0)

# if building is already equipped with ASHP, set the ASHP purchase cost to 0
if row['already_ASHP']:
    building_specific_config.set_key(f'locations.{building.name}.techs.ASHP.costs.monetary.purchase', 0)
    building_specific_config.set_key(f'locations.{building.name}.techs.ASHP.costs.monetary.energy_cap', 0)

# if building is not rebuilt, set GSHP and ASHP costs higher
if not row['is_rebuilt']:
    if not row['already_GSHP']:
        building_specific_config.set_key(f'locations.{building.name}.techs.GSHP_heat.costs.monetary.purchase', 39934)
        building_specific_config.set_key(f'locations.{building.name}.techs.GSHP_heat.costs.monetary.energy_cap', 1316)
    if not row['already_ASHP']:
        building_specific_config.set_key(f'locations.{building.name}.techs.ASHP.costs.monetary.purchase', 18086)
        building_specific_config.set_key(f'locations.{building.name}.techs.ASHP.costs.monetary.energy_cap', 1360)

    # if building is not new, delete the GSHP technology keys
if not row['is_new']:
    building_specific_config.del_key(f'locations.{building.name}.techs.GSHP_heat')
    building_specific_config.del_key(f'locations.{building.name}.techs.GSHP_cooling')
    building_specific_config.del_key(f'locations.{building.name}.techs.geothermal_boreholes')

# # test: delete ASHP
# building_specific_config.del_key(f'locations.{building.name}.techs.ASHP')        

model = calliope.Model(building_specific_config, timeseries_dataframes=dict_timeseries_df)
# model.to_lp('./result_site_specific_2/'+building.name+'.lp')
model.run()
# model.save_commented_model_yaml('./result_site_specific_2/'+building.name+'.yaml')
# # save to result_per_building, which is a subfolder of current working directory
# model.to_netcdf(path='./result_site_specific_2/'+building.name+'.nc')
# # calculate how many percent of the buildings are done, including the current one and its name
# # calculate how many .nc files are in the subfolder, but some files are not .nc files, so len(os.listdir()) is not accurate
# number_of_nc_files = len(glob.glob('./result_site_specific_2/*.nc'))
# print('including '+building.name+', '+str(round(number_of_nc_files/len(zone_df.index)*100, 2))+'% of the buildings are done')

In [ ]:
model_tech_cost = model.get_formatted_array('cost').sel(locs='B302030807').to_pandas().transpose()
model_energy_cap = model.get_formatted_array('energy_cap').to_pandas().iloc[0]

In [ ]:
# model = calliope.read_netcdf('./result_site_specific_2/B302030807.nc')
model_geostore_prod = model.get_formatted_array('carrier_prod').sel(carriers='geothermal_storage', locs='B302030807').to_pandas().transpose()
model_geostore_con = model.get_formatted_array('carrier_con').sel(carriers='geothermal_storage', locs='B302030807').to_pandas().transpose()
model_geostore_all = model_geostore_prod.merge(model_geostore_con, left_index=True, right_index=True)

model_sh_prod = model.get_formatted_array('carrier_prod').sel(carriers='heat', locs='B302030807').to_pandas().transpose()
model_sh_con = model.get_formatted_array('carrier_con').sel(carriers='heat', locs='B302030807').to_pandas().transpose()
model_sh_all = model_sh_prod.merge(model_sh_con, left_index=True, right_index=True)

model_el_prod = model.get_formatted_array('carrier_prod').sel(carriers='electricity', locs='B302030807').to_pandas().transpose()
model_el_con = model.get_formatted_array('carrier_con').sel(carriers='electricity', locs='B302030807').to_pandas().transpose()
model_el_all = model_el_prod.merge(model_el_con, left_index=True, right_index=True)

In [ ]:
# plot the model_geostore_all, with x axis as timestep, and four ys: geothermal_boreholes_x, geothermal_boreholes_y, GSHP_heat_y, GSHP_cooling_x
plt.plot(model_geostore_all.index, model_geostore_all['geothermal_boreholes_x'], color='red', alpha=0.5)
plt.plot(model_geostore_all.index, model_geostore_all['geothermal_boreholes_y'], color='blue', alpha=0.5)
plt.plot(model_geostore_all.index, model_geostore_all['GSHP_heat_y'], color='green', alpha=0.5)
plt.plot(model_geostore_all.index, model_geostore_all['GSHP_cooling_x'], color='orange', alpha=0.5)
plt.legend(['geothermal_boreholes_x', 'geothermal_boreholes_y', 'GSHP_heat_y', 'GSHP_cooling_x'], loc='upper right')
plt.title('geothermal storage used by technologies')
plt.ylabel('energy [kWh]')
plt.show()

plt.plot(model_sh_all.index, model_sh_all['demand_space_heating'], color='red', alpha=0.5)
plt.plot(model_sh_all.index, model_sh_all['GSHP_heat_x'], color='green', alpha=0.5)
plt.plot(model_sh_all.index, model_sh_all['wood_boiler_heat_x'], color='blue', alpha=0.5)
plt.legend(['demand_space_heating', 'GSHP_heat_x', 'wood_boiler_heat_x'], loc='upper right')
plt.title('space heating demand and technologies')
plt.ylabel('energy [kWh]')
plt.show()

# plot in model_el_all, with x axis as timestep, and ys: PV, battery_x, battery_y, grid, GSHP_heat_y, GSHP_cooling_y
plt.plot(model_el_all.index, model_el_all['demand_electricity'], color='red', alpha=0.5)
plt.plot(model_el_all.index, model_el_all['PV'], color='green', alpha=0.5)
plt.plot(model_el_all.index, model_el_all['battery_x'], color='blue', alpha=0.5)
plt.plot(model_el_all.index, model_el_all['battery_y'], color='orange', alpha=0.5)
plt.plot(model_el_all.index, model_el_all['grid'], color='purple', alpha=0.5)
plt.legend(['demand_electricity', 'PV', 'battery_x', 'battery_y', 'grid'], loc='upper right')
plt.title('electricity demand and technologies')
plt.ylabel('energy [kWh]')
plt.show()